In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from pathlib import Path
import kagglehub

# ---------------------------------------------------
# 1. Load Dataset
# ---------------------------------------------------
path = kagglehub.dataset_download("shahriar26s/banana-ripeness-classification-dataset")
dataset_root = Path(path) / "Banana Ripeness Classification Dataset"

train_dir = dataset_root / "train"
val_dir   = dataset_root / "valid"
test_dir  = dataset_root / "test"

# ---------------------------------------------------
# 2. Image Generators
# ---------------------------------------------------
img_size = (160, 160)               # smaller = faster
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1/255,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1/255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

# ---------------------------------------------------
# 3. Load Pretrained MobileNetV2
# ---------------------------------------------------
base_model = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(160, 160, 3)
)

base_model.trainable = False   # freeze weights → very fast training

# ---------------------------------------------------
# 4. Build Final Model
# ---------------------------------------------------
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ---------------------------------------------------
# 5. Train (very fast)
# ---------------------------------------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5    # only 5 needed
)

# ---------------------------------------------------
# 6. Evaluate
# ---------------------------------------------------
test_gen = ImageDataGenerator(rescale=1/255)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc)

# ---------------------------------------------------
# 7. Save Model
# ---------------------------------------------------
model.save("banana_mobilenetv2.h5")
print("Model saved!")


Using Colab cache for faster access to the 'banana-ripeness-classification-dataset' dataset.
Found 11793 images belonging to 4 classes.
Found 1123 images belonging to 4 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        40,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,299,108 (8.77 MB)

 Trainable params: 41,124 (160.64 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 273s 725ms/step - accuracy: 0.8683 - loss: 0.3627 - val_accuracy: 0.9635 - val_loss: 0.1318
Epoch 2/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 260s 703ms/step - accuracy: 0.9622 - loss: 0.1144 - val_accuracy: 0.9590 - val_loss: 0.1402
Epoch 3/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 261s 708ms/step - accuracy: 0.9682 - loss: 0.0922 - val_accuracy: 0.9679 - val_loss: 0.1093
Epoch 4/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 261s 708ms/step - accuracy: 0.9762 - loss: 0.0694 - val_accuracy: 0.9688 - val_loss: 0.1259
Epoch 5/5
369/369 ━━━━━━━━━━━━━━━━━━━━ 262s 709ms/step - accuracy: 0.9756 - loss: 0.0675 - val_accuracy: 0.9697 - val_loss: 0.1034
Found 562 images belonging to 4 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 643ms/step - accuracy: 0.9653 - loss: 0.1049


Test Accuracy: 0.9715302586555481
Model saved!
